In [81]:
import codecs
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime

Load data in Tensorflow.

In [82]:
root = "/Users/pasquale/git/recommender/"
training_data_folder = [
    '%straining_data/web-radio/output/list' % root,
    '%straining_data/spotify/output/playlists/list' % root,
    '%straining_data/concerts/output/list/Philharmonie_de_Paris' % root,
    '%straining_data/concerts/output/list/Radio_France' % root
    ]
embDir = '/Users/pasquale/git/music-embeddings'
what = 'artist'

uri_file = '%s/%s.emb.u' % (embDir, what)
vector_file = '%s/%s.emb.v' % (embDir, what)
header_file = '%s/%s.emb.h' % (embDir, what)

In [83]:
# load embeddings
vectors = np.array([line.strip().split(' ') for line in codecs.open(vector_file, 'r', 'utf-8')])
heads = np.array([line.strip() for line in codecs.open(header_file, 'r', 'utf-8')])
uris = np.array([line.strip() for line in codecs.open(uri_file, 'r', 'utf-8')])

In [84]:
head_label = heads[0].split()
head_val = heads[1].split()
head_dim = []
for i in range(0, len(head_val)):
    print(head_label[i] + ' -> ' + head_val[i])
    for j in range(0, int(head_val[i])):
        head_dim.append(head_label[i])
        
# head_dim.append('fake')
# head_dim.append('fake')
# head_dim.append('fake')
# head_dim

mop -> 80
birth_date -> 1
death_date -> 1
casting -> 80
genre -> 80
key -> 100


In [85]:
# load training
train_array = []
for folder in training_data_folder:
    training_folder = '%s/%s' % (folder, what)
    for f in os.listdir(training_folder):
        file= '%s/%s' % (training_folder, f)
        train_array.append(np.array([line.strip() for line in codecs.open(file, 'r', 'utf-8')]))

len(train_array)

849

Data pre-processing

In [89]:
def get_embs(x):
    # uri to embedding
    v = vectors[np.argwhere(uris == x)]
    if v.size == 0:
        result = -2. * np.ones_like(vectors[0], dtype=np.float32)
    else:
        result = np.array(v[0][0])
    return result.astype(np.float32)

In [137]:
minibatches_len = 8

In [138]:
training_vector_all = []
for t in train_array:
    if(len(t) < minibatches_len):
        continue
    training_vector_all.append(np.array([get_embs(xi) for xi in t]))

I divide the playlists in minibatches of 2X songs (X seeds and X targets)

In [252]:
training_vector = []

for t in training_vector_all:
    for x in range(1, len(t) - minibatches_len):
        training_vector.append(t[0:minibatches_len])
        
training_vector = np.array(training_vector)
training_vector.shape

(4254, 8, 342)

In [296]:
# train_label[399]

In [297]:
def smart_mean(a):
    b = np.where(a < -1., 0., a)
    s = np.sum(b, axis=1)
    c = np.count_nonzero(b, axis=1)
    d = np.divide(s, c)
    return np.where(np.isnan(d), 0, d)

Split test and train

In [255]:
train, test = train_test_split(training_vector, train_size=0.7)

train_seed, train_target = np.split(train, 2, axis=1)
train_vector = smart_mean(train_seed)
train_label = smart_mean(train_target)

test_seed, test_target = np.split(test, 2, axis=1)
test_vector = smart_mean(test_seed)
test_label = smart_mean(test_target)

/Users/pasquale/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/Users/pasquale/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [226]:
print('Train')
print(train_vector.shape)
print(train_label.shape)
print('Test')
print(test_vector.shape)
print(test_label.shape)

Train
(2977, 342)
(2977, 342)
Test
(1277, 342)
(1277, 342)


In [268]:
# Parameters
learning_rate = 0.1
num_steps = 10000
batch_size = 128
display_step = 1

In [269]:
# Network Parameters
n_hidden_1 = 256  # 1st layer number of neurons
n_hidden_2 = 256  # 2nd layer number of neurons
num_input = train_vector[0].size
num_output = train_label[0].size

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input], name="X")
Y = tf.placeholder(tf.float32, [None, num_output], name="Y")

Neural network

In [270]:
# Create model
def neural_net(x):
    with tf.name_scope('hidden_1') as scope:
        # Hidden fully connected layer with 256 neurons
        w1 = tf.Variable(tf.random_normal([num_input, n_hidden_1]), name='w')
        b1 = tf.Variable(tf.random_normal([n_hidden_1]), name='b')
        layer_1 = tf.add(tf.matmul(x, w1), b1, name='o')
    with tf.name_scope('hidden_2') as scope:
        # Hidden fully connected layer with 256 neurons
        w2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='w')
        b2 = tf.Variable(tf.random_normal([n_hidden_2]), name='b')
        layer_2 = tf.add(tf.matmul(layer_1, w2), b2, name='o')
    with tf.name_scope('out_layer') as scope:
        # Output fully connected layer with a neuron for each class
        wo = tf.Variable(tf.random_normal([n_hidden_2, num_output]), name='w')
        bo = tf.Variable(tf.random_normal([num_output]), name='b')
        out_layer = tf.add(tf.matmul(layer_2, wo), bo, name="o")
    return out_layer
#         return tf.nn.softmax(out_layer, name = 'y')

In [271]:
def weighted_l2(a, b, w):
    with tf.name_scope('weighted_l2') as scope:
        # https://stackoverflow.com/a/8861999/1218213
        q = tf.subtract(a, b, name="q")
        # return np.sqrt((w * q * q).sum())
        pow_q = tf.cast(tf.pow(q, 2), tf.float32, name="q-power")

        return tf.reduce_sum(tf.multiply(w, pow_q), axis=1, name="o", keepdims=True)

In [272]:
def compute_penalty(expected, taken, total):
    with tf.name_scope('penalty') as scope:
        penalty = tf.divide(tf.subtract(expected, taken), total)
        return tf.cast(penalty, tf.float32)

In [273]:
# Construct model
logits = neural_net(X)

In [274]:
logits.shape

TensorShape([Dimension(None), Dimension(342)])

In [298]:
# Define loss and optimizer
# loss_op = MSE
loss_op = tf.reduce_mean(tf.square(tf.subtract(logits, Y)), name='loss_op')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='optimizer')
train_op = optimizer.minimize(loss_op)
error_summary = tf.summary.scalar('error', loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(logits, Y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge([error_summary, accuracy_summary])

In [299]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [300]:
def next_batch(num, data, labels):
    """
    Return a total of `num` random samples and labels. 
    """
    idx = np.arange(0, len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = data[idx]
    labels_shuffle = labels[idx]
    return data_shuffle, labels_shuffle

In [ ]:
sess = tf.Session()
now = datetime.now()
log_dir = '%straining/output/%s' % (root, now.strftime("%Y%m%d-%H%M%S"))
writer = tf.summary.FileWriter(log_dir, sess.graph)

# Run the initializer
sess.run(init)

print("Start learning")
for step in range(1, num_steps + 1):
    batch_x, batch_y = next_batch(batch_size, train_vector, train_label)
#     print(batch_y)
    # Run optimization op (backprop)
    sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
    if step % display_step == 0 or step == 1:
        # Calculate batch loss and accuracy
        preds, summary, loss, acc = sess.run([logits, merged, loss_op, accuracy],
                                                feed_dict={X: batch_x, Y: batch_y})

        writer.add_summary(summary, step)

        print("Step " + str(step) + ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))
#         print("Predictions %s VS %s" % (preds[0][0], batch_y[0][0]))
        np.set_printoptions(precision=2)

print("Optimization Finished!")

print("Testing Accuracy:",
      sess.run(accuracy, feed_dict={X: test_vector, Y: test_label}))
writer.flush()
writer.close()



Start learning
Step 1, Minibatch Loss= 586542.2500, Training Accuracy= 0.000
Step 2, Minibatch Loss= 636334.6250, Training Accuracy= 0.000
Step 3, Minibatch Loss= 328609.6875, Training Accuracy= 0.000
Step 4, Minibatch Loss= 109718.1953, Training Accuracy= 0.000
Step 5, Minibatch Loss= 145589.8906, Training Accuracy= 0.000
Step 6, Minibatch Loss= 92217.3750, Training Accuracy= 0.000
Step 7, Minibatch Loss= 82512.9844, Training Accuracy= 0.000
Step 8, Minibatch Loss= 78164.0781, Training Accuracy= 0.000
Step 9, Minibatch Loss= 75039.8047, Training Accuracy= 0.000
Step 10, Minibatch Loss= 53330.7031, Training Accuracy= 0.000
Step 11, Minibatch Loss= 35251.9766, Training Accuracy= 0.000
Step 12, Minibatch Loss= 38458.5664, Training Accuracy= 0.000
Step 13, Minibatch Loss= 51147.0977, Training Accuracy= 0.000
Step 14, Minibatch Loss= 44290.9883, Training Accuracy= 0.000
Step 15, Minibatch Loss= 26655.2598, Training Accuracy= 0.000
Step 16, Minibatch Loss= 18531.7129, Training Accuracy= 0.0

Step 148, Minibatch Loss= 423.4542, Training Accuracy= 0.000
Step 149, Minibatch Loss= 399.6558, Training Accuracy= 0.000
Step 150, Minibatch Loss= 421.3452, Training Accuracy= 0.000
Step 151, Minibatch Loss= 491.2576, Training Accuracy= 0.000
Step 152, Minibatch Loss= 399.0494, Training Accuracy= 0.000
Step 153, Minibatch Loss= 454.1473, Training Accuracy= 0.000
Step 154, Minibatch Loss= 454.3094, Training Accuracy= 0.000
Step 155, Minibatch Loss= 377.3164, Training Accuracy= 0.000
Step 156, Minibatch Loss= 567.5637, Training Accuracy= 0.000
Step 157, Minibatch Loss= 429.3218, Training Accuracy= 0.000
Step 158, Minibatch Loss= 406.5860, Training Accuracy= 0.000
Step 159, Minibatch Loss= 445.5957, Training Accuracy= 0.000
Step 160, Minibatch Loss= 544.9349, Training Accuracy= 0.000
Step 161, Minibatch Loss= 435.1767, Training Accuracy= 0.000
Step 162, Minibatch Loss= 431.1855, Training Accuracy= 0.000
Step 163, Minibatch Loss= 380.3553, Training Accuracy= 0.000
Step 164, Minibatch Loss

Step 309, Minibatch Loss= 192.3443, Training Accuracy= 0.000
Step 310, Minibatch Loss= 214.6926, Training Accuracy= 0.000
Step 311, Minibatch Loss= 193.5460, Training Accuracy= 0.000
Step 312, Minibatch Loss= 214.0906, Training Accuracy= 0.000
Step 313, Minibatch Loss= 231.3479, Training Accuracy= 0.000
Step 314, Minibatch Loss= 160.1876, Training Accuracy= 0.000
Step 315, Minibatch Loss= 223.1429, Training Accuracy= 0.000
Step 316, Minibatch Loss= 191.3077, Training Accuracy= 0.000
Step 317, Minibatch Loss= 202.0752, Training Accuracy= 0.000
Step 318, Minibatch Loss= 210.9043, Training Accuracy= 0.000
Step 319, Minibatch Loss= 195.7049, Training Accuracy= 0.000
Step 320, Minibatch Loss= 201.1633, Training Accuracy= 0.000
Step 321, Minibatch Loss= 180.5786, Training Accuracy= 0.000
Step 322, Minibatch Loss= 188.0965, Training Accuracy= 0.000
Step 323, Minibatch Loss= 170.3281, Training Accuracy= 0.000
Step 324, Minibatch Loss= 170.7802, Training Accuracy= 0.000
Step 325, Minibatch Loss

Step 462, Minibatch Loss= 109.1729, Training Accuracy= 0.000
Step 463, Minibatch Loss= 121.9055, Training Accuracy= 0.000
Step 464, Minibatch Loss= 113.6557, Training Accuracy= 0.000
Step 465, Minibatch Loss= 111.5426, Training Accuracy= 0.000
Step 466, Minibatch Loss= 128.7601, Training Accuracy= 0.000
Step 467, Minibatch Loss= 110.7627, Training Accuracy= 0.000
Step 468, Minibatch Loss= 96.0053, Training Accuracy= 0.000
Step 469, Minibatch Loss= 111.1928, Training Accuracy= 0.000
Step 470, Minibatch Loss= 131.8285, Training Accuracy= 0.000
Step 471, Minibatch Loss= 94.1236, Training Accuracy= 0.000
Step 472, Minibatch Loss= 110.4928, Training Accuracy= 0.000
Step 473, Minibatch Loss= 117.3497, Training Accuracy= 0.000
Step 474, Minibatch Loss= 105.6119, Training Accuracy= 0.000
Step 475, Minibatch Loss= 121.7638, Training Accuracy= 0.000
Step 476, Minibatch Loss= 101.9188, Training Accuracy= 0.000
Step 477, Minibatch Loss= 99.2958, Training Accuracy= 0.000
Step 478, Minibatch Loss= 9

Step 618, Minibatch Loss= 90.7785, Training Accuracy= 0.000
Step 619, Minibatch Loss= 78.9304, Training Accuracy= 0.000
Step 620, Minibatch Loss= 95.2037, Training Accuracy= 0.000
Step 621, Minibatch Loss= 67.7971, Training Accuracy= 0.000
Step 622, Minibatch Loss= 85.0441, Training Accuracy= 0.000
Step 623, Minibatch Loss= 83.6742, Training Accuracy= 0.000
Step 624, Minibatch Loss= 79.8811, Training Accuracy= 0.000
Step 625, Minibatch Loss= 80.3873, Training Accuracy= 0.000
Step 626, Minibatch Loss= 76.5604, Training Accuracy= 0.000
Step 627, Minibatch Loss= 78.8792, Training Accuracy= 0.000
Step 628, Minibatch Loss= 80.9817, Training Accuracy= 0.000
Step 629, Minibatch Loss= 66.5125, Training Accuracy= 0.000
Step 630, Minibatch Loss= 83.0260, Training Accuracy= 0.000
Step 631, Minibatch Loss= 75.6567, Training Accuracy= 0.000
Step 632, Minibatch Loss= 72.5815, Training Accuracy= 0.000
Step 633, Minibatch Loss= 79.4020, Training Accuracy= 0.000
Step 634, Minibatch Loss= 85.6347, Train

...

Step 2106, Minibatch Loss= 14.3213, Training Accuracy= 0.000
Step 2107, Minibatch Loss= 18.9798, Training Accuracy= 0.000
Step 2108, Minibatch Loss= 19.3068, Training Accuracy= 0.000
Step 2109, Minibatch Loss= 18.7595, Training Accuracy= 0.000
Step 2110, Minibatch Loss= 20.6002, Training Accuracy= 0.000
Step 2111, Minibatch Loss= 21.5668, Training Accuracy= 0.000
Step 2112, Minibatch Loss= 20.7869, Training Accuracy= 0.000
Step 2113, Minibatch Loss= 21.4897, Training Accuracy= 0.000
Step 2114, Minibatch Loss= 25.4683, Training Accuracy= 0.000
Step 2115, Minibatch Loss= 30.3126, Training Accuracy= 0.000
Step 2116, Minibatch Loss= 35.3835, Training Accuracy= 0.000
Step 2117, Minibatch Loss= 37.4244, Training Accuracy= 0.000
Step 2118, Minibatch Loss= 49.3372, Training Accuracy= 0.000
Step 2119, Minibatch Loss= 54.8156, Training Accuracy= 0.000
Step 2120, Minibatch Loss= 45.8620, Training Accuracy= 0.000
Step 2121, Minibatch Loss= 47.3135, Training Accuracy= 0.000
Step 2122, Minibatch Los

Step 2253, Minibatch Loss= 20.8290, Training Accuracy= 0.000
Step 2254, Minibatch Loss= 21.2239, Training Accuracy= 0.000
Step 2255, Minibatch Loss= 22.1040, Training Accuracy= 0.000
Step 2256, Minibatch Loss= 20.4190, Training Accuracy= 0.000
Step 2257, Minibatch Loss= 15.9397, Training Accuracy= 0.000
Step 2258, Minibatch Loss= 13.9029, Training Accuracy= 0.000
Step 2259, Minibatch Loss= 16.6361, Training Accuracy= 0.000
Step 2260, Minibatch Loss= 22.1076, Training Accuracy= 0.000
Step 2261, Minibatch Loss= 26.7320, Training Accuracy= 0.000
Step 2262, Minibatch Loss= 31.1383, Training Accuracy= 0.000
Step 2263, Minibatch Loss= 30.3859, Training Accuracy= 0.000
Step 2264, Minibatch Loss= 47.3211, Training Accuracy= 0.000
Step 2265, Minibatch Loss= 78.3590, Training Accuracy= 0.000
Step 2266, Minibatch Loss= 77.4422, Training Accuracy= 0.000
Step 2267, Minibatch Loss= 101.2181, Training Accuracy= 0.000
Step 2268, Minibatch Loss= 66.9499, Training Accuracy= 0.000
Step 2269, Minibatch Lo

Step 2400, Minibatch Loss= 18.5223, Training Accuracy= 0.000
Step 2401, Minibatch Loss= 17.1007, Training Accuracy= 0.000
Step 2402, Minibatch Loss= 18.8923, Training Accuracy= 0.000
Step 2403, Minibatch Loss= 25.3404, Training Accuracy= 0.000
Step 2404, Minibatch Loss= 23.9113, Training Accuracy= 0.000
Step 2405, Minibatch Loss= 23.5092, Training Accuracy= 0.000
Step 2406, Minibatch Loss= 26.7104, Training Accuracy= 0.000
Step 2407, Minibatch Loss= 23.5047, Training Accuracy= 0.000
Step 2408, Minibatch Loss= 32.0369, Training Accuracy= 0.000
Step 2409, Minibatch Loss= 28.5523, Training Accuracy= 0.000
Step 2410, Minibatch Loss= 38.5725, Training Accuracy= 0.000
Step 2411, Minibatch Loss= 45.2412, Training Accuracy= 0.000
Step 2412, Minibatch Loss= 51.1999, Training Accuracy= 0.000
Step 2413, Minibatch Loss= 57.8478, Training Accuracy= 0.000
Step 2414, Minibatch Loss= 48.9937, Training Accuracy= 0.000
Step 2415, Minibatch Loss= 44.9434, Training Accuracy= 0.000
Step 2416, Minibatch Los

Step 2556, Minibatch Loss= 13.9793, Training Accuracy= 0.000
Step 2557, Minibatch Loss= 20.2420, Training Accuracy= 0.000
Step 2558, Minibatch Loss= 25.3344, Training Accuracy= 0.000
Step 2559, Minibatch Loss= 28.1271, Training Accuracy= 0.000
Step 2560, Minibatch Loss= 31.1927, Training Accuracy= 0.000
Step 2561, Minibatch Loss= 38.5976, Training Accuracy= 0.000
Step 2562, Minibatch Loss= 58.3671, Training Accuracy= 0.000
Step 2563, Minibatch Loss= 81.9015, Training Accuracy= 0.000
Step 2564, Minibatch Loss= 111.0300, Training Accuracy= 0.000
Step 2565, Minibatch Loss= 109.7868, Training Accuracy= 0.000
Step 2566, Minibatch Loss= 65.3842, Training Accuracy= 0.000
Step 2567, Minibatch Loss= 27.2977, Training Accuracy= 0.000
Step 2568, Minibatch Loss= 22.2375, Training Accuracy= 0.000
Step 2569, Minibatch Loss= 33.9197, Training Accuracy= 0.000
Step 2570, Minibatch Loss= 31.8480, Training Accuracy= 0.000
Step 2571, Minibatch Loss= 13.8211, Training Accuracy= 0.000
Step 2572, Minibatch L

Step 2714, Minibatch Loss= 12.3793, Training Accuracy= 0.000
Step 2715, Minibatch Loss= 11.6502, Training Accuracy= 0.000
Step 2716, Minibatch Loss= 9.4240, Training Accuracy= 0.000
Step 2717, Minibatch Loss= 8.4416, Training Accuracy= 0.000
Step 2718, Minibatch Loss= 7.9710, Training Accuracy= 0.000
Step 2719, Minibatch Loss= 7.9829, Training Accuracy= 0.000
Step 2720, Minibatch Loss= 7.6617, Training Accuracy= 0.000
Step 2721, Minibatch Loss= 9.1950, Training Accuracy= 0.000
Step 2722, Minibatch Loss= 10.7388, Training Accuracy= 0.000
Step 2723, Minibatch Loss= 8.6266, Training Accuracy= 0.000
Step 2724, Minibatch Loss= 8.7008, Training Accuracy= 0.000
Step 2725, Minibatch Loss= 7.5815, Training Accuracy= 0.000
Step 2726, Minibatch Loss= 8.5945, Training Accuracy= 0.000
Step 2727, Minibatch Loss= 7.5578, Training Accuracy= 0.000
Step 2728, Minibatch Loss= 7.6302, Training Accuracy= 0.000
Step 2729, Minibatch Loss= 8.5042, Training Accuracy= 0.000
Step 2730, Minibatch Loss= 9.3972, Tr

In [ ]:
verdi = get_embs('http://data.doremus.org/artist/b82c0771-5280-39af-ad2e-8ace2f4ebda3')
rossini = get_embs('http://data.doremus.org/artist/c5d5735c-1095-3ed4-a20f-1208ab9567f9')
rossini_pp = get_embs('http://data.doremus.org/artist/32c2b0ff-35f1-3e65-b0ca-34aaf35f3d50')
beethoven = get_embs('http://data.doremus.org/artist/6963af5e-b126-3d40-a84b-97e0b78f5452')
mozart = get_embs('http://data.doremus.org/artist/4802a043-23bb-3b8d-a443-4a3bd22ccc63')
ravel = get_embs('http://data.doremus.org/artist/1b1205f9-b99a-3bb8-ba77-256689af2e00')
gershwin = get_embs('http://data.doremus.org/artist/5b2ec204-a456-3aa2-8ac7-25305464add8')
coltrane = get_embs('http://data.doremus.org/artist/5425efed-002f-3638-a7b0-ad379a2bf63d')

preds = sess.run(logits, feed_dict={X: rossini.reshape(1,342), Y: test_label[0].reshape(1,342)})
print(preds)
